# 第 11 章 时间序列

In [11]:
import pandas as pd
import numpy as np

In [19]:
#时间序列也可以是不定期的，没有固定的时间单位或单位之间的 偏移量。
#时间序列数据的意义取决于具体的应用场景，主要有以下几种:

In [ ]:
 时间戳(timestamp)，特定的时刻。
 固定时期(period)，如 2007 年 1 月或 2010 年全年。
 时间间隔(interval)，由起始和结束时间戳表示。时期(period)可以被看做间隔
(interval)的特例。
 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入
   烤箱时起，每秒钟饼干的直径。

# 11.1 日期和时间数据类型及工具 P337

In [ ]:
#Python 标准库包含用于日期(date)和时间(time)数据的数据类型，而且还 有日历方面的功能。

In [ ]:
#datetime.datetime(也可以简写为 datetime)是用得最多的数据类型:

In [20]:
from datetime import datetime

In [21]:
now = datetime.now()
now

datetime.datetime(2019, 8, 31, 12, 33, 28, 273231)

In [22]:
now.year, now.month, now.day

(2019, 8, 31)

In [ ]:
#datetime 以毫秒形式存储日期和时间。
#timedelta 表示两个 datetime 对象之间 的时间差：

In [23]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(days=926, seconds=56700)

In [24]:
delta.days

926

In [25]:
delta.seconds

56700

In [ ]:
#可以给 datetime 对象加上(或减去)一个或多个 timedelta，这样会产生一个 新对象

In [26]:
from datetime import timedelta

In [27]:
start = datetime(2011, 1, 7)

In [28]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [30]:
2 * timedelta(12)

datetime.timedelta(days=24)

In [31]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

## 字符串和 datetime 的相互转换


In [ ]:
#利用 str 或 strftime 方法(传入一个格式化字符串)，datetime 对象和 pandas 的 Timestamp 对象(稍后就会介绍)可以被格式化为字符串:

In [66]:
stamp = datetime(2011, 1, 3)
stamp    #类型： datetime.datetime

datetime.datetime(2011, 1, 3, 0, 0)

In [34]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [35]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

In [ ]:
#datetime.strptime 可以用这些格式化编码将字符串转换为日期:

In [37]:
value = '2011-01-03'

In [38]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [39]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

In [ ]:
#datetime.strptime 是通过已知格式进行日期解析的最佳方式。

In [ ]:
#你可以用 dateutil 这个第三方包中的 parser.parse 方法(pandas 中已经 自动安装好了):

In [40]:
from dateutil.parser import parse
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [ ]:
#dateutil 可以解析几乎所有人类能够理解的日期表示形式:

In [41]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

In [ ]:
#在国际通用的格式中，日出现在月的前面很普遍，传入 dayfirst=True 即可解 决这个问题:

In [42]:
parse('6/12/2011', dayfirst=True) #需要实际看清楚日期的位置

datetime.datetime(2011, 12, 6, 0, 0)

In [ ]:
#pandas 通常是用于处理成组日期的，不管这些日期是 DataFrame 的轴索引还是 列。

In [ ]:
#to_datetime 方法可以解析多种不同的日期表示形式。对标准日期格式 (如 ISO8601)的解析非常快:

In [43]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

In [44]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

In [ ]:
#它还可以处理缺失值(None、空字符串等):

In [45]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [46]:
idx[2]  #NaT(Not a Time)是 pandas 中时间戳数据的 null 值。`

NaT

In [47]:
pd.isnull(idx)

array([False, False,  True])

# 11.2 时间序列基础 P341

In [7]:
#pandas 最基本的时间序列类型就是以时间戳(通常以 Python 字符串或 datatime 对象表示)为索引的 Series:

In [12]:
from datetime import datetime

In [13]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [14]:
#这些 datetime 对象实际上是被放在一个 DatetimeIndex 中的:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
#跟其他 Series 一样，不同索引的时间序列之间的算术运算会自动按日期对齐:

In [69]:
ts[::2]

2011-01-02    0.517332
2011-01-07   -0.357989
2011-01-10    0.347589
dtype: float64

In [15]:
ts + ts[::2]  #ts[::2] 是每隔两个取一个。

2011-01-02    1.034664
2011-01-05         NaN
2011-01-07   -0.715977
2011-01-08         NaN
2011-01-10    0.695178
2011-01-12         NaN
dtype: float64

In [16]:
#pandas 用 NumPy 的 datetime64 数据类型以纳秒形式存储时间戳:
ts.index.dtype

dtype('<M8[ns]')

In [17]:
#DatetimeIndex 中的各个标量值是 pandas 的 Timestamp 对象:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

## 索引、选取、子集构造

In [70]:
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [ ]:
#当你根据标签索引选取数据时，时间序列和其它的 pandas.Series 很像:

In [49]:
stamp = ts.index[2]
stamp

Timestamp('2011-01-07 00:00:00')

In [51]:
ts[stamp]        #对应去索引为'2011-01-07 00:00:00'的值

-0.3579887488721009

In [ ]:
#还有一种更为方便的用法:传入一个可以被解释为日期的字符串:

In [52]:
ts['1/10/2011']

0.3475891606481819

In [53]:
ts['20110110']

0.3475891606481819

In [71]:
#对于较长的时间序列，只需传入“年”或“年月”即可轻松选取数据的切片:
longer_ts = pd.Series(np.random.randn(1000),
                      index=pd.date_range('1/1/2000',
periods=1000))
longer_ts

2000-01-01    0.343674
2000-01-02    0.778471
2000-01-03    0.875276
2000-01-04   -0.405060
2000-01-05   -0.719798
2000-01-06    0.196924
2000-01-07   -0.971296
2000-01-08   -0.797292
2000-01-09    0.749376
2000-01-10   -0.666872
2000-01-11   -0.106474
2000-01-12   -1.180442
2000-01-13    0.085379
2000-01-14    0.738840
2000-01-15    0.661143
2000-01-16   -1.126663
2000-01-17    0.104495
2000-01-18   -1.328034
2000-01-19   -1.031598
2000-01-20   -0.349940
2000-01-21   -0.445204
2000-01-22    0.505863
2000-01-23   -0.098758
2000-01-24    0.041616
2000-01-25    0.409775
2000-01-26    0.013029
2000-01-27   -0.196724
2000-01-28   -1.818619
2000-01-29    0.164077
2000-01-30   -0.772608
                ...   
2002-08-28    0.495989
2002-08-29    1.381707
2002-08-30    2.132679
2002-08-31   -0.897966
2002-09-01    1.206236
2002-09-02   -0.325423
2002-09-03   -0.378993
2002-09-04   -1.171323
2002-09-05   -0.539013
2002-09-06   -0.082567
2002-09-07   -0.322164
2002-09-08    1.208663
2002-09-09 

In [72]:
longer_ts['2001']    #字符串“2001”被解释成年，并根据它选取时间区间。

2001-01-01   -0.580101
2001-01-02    1.139566
2001-01-03    0.934095
2001-01-04    1.459325
2001-01-05    0.939479
2001-01-06    0.560135
2001-01-07   -0.037009
2001-01-08   -0.041499
2001-01-09    1.702968
2001-01-10    0.074819
2001-01-11    0.677975
2001-01-12   -1.546243
2001-01-13    0.648762
2001-01-14   -1.223575
2001-01-15    1.420048
2001-01-16    0.337703
2001-01-17   -1.453696
2001-01-18   -0.339018
2001-01-19   -0.425005
2001-01-20   -2.031672
2001-01-21    1.556392
2001-01-22   -0.761777
2001-01-23    0.920782
2001-01-24   -0.053243
2001-01-25   -0.502454
2001-01-26    1.113653
2001-01-27    0.145992
2001-01-28   -0.423661
2001-01-29   -0.059715
2001-01-30   -0.759990
                ...   
2001-12-02   -0.593445
2001-12-03   -0.914355
2001-12-04   -1.760360
2001-12-05    1.795751
2001-12-06    1.674011
2001-12-07   -0.372508
2001-12-08    0.456434
2001-12-09    2.790975
2001-12-10    0.643589
2001-12-11   -0.090345
2001-12-12    1.052999
2001-12-13   -0.086699
2001-12-14 

In [73]:
longer_ts['2001-05']   #指定月选取时间区间

2001-05-01    0.363887
2001-05-02   -1.319376
2001-05-03   -0.843747
2001-05-04   -1.848581
2001-05-05    0.302303
2001-05-06    0.181640
2001-05-07   -0.044887
2001-05-08   -0.458012
2001-05-09   -1.231411
2001-05-10    0.611642
2001-05-11    1.000950
2001-05-12    0.527681
2001-05-13    0.621344
2001-05-14    0.176120
2001-05-15    0.319491
2001-05-16   -1.797881
2001-05-17   -0.665354
2001-05-18   -1.735888
2001-05-19    0.888787
2001-05-20    0.361085
2001-05-21    0.217500
2001-05-22   -0.362782
2001-05-23    1.906739
2001-05-24   -0.887970
2001-05-25   -0.549737
2001-05-26    0.759317
2001-05-27   -0.045476
2001-05-28    1.688544
2001-05-29    0.629789
2001-05-30    1.974604
2001-05-31    0.970394
Freq: D, dtype: float64

In [ ]:
#datetime 对象也可以进行切片:

In [74]:
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [75]:
ts[datetime(2011, 1, 7):]

2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [ ]:
#由于大部分时间序列数据都是按照时间先后排序的，因此你也可以用不存在于
#该时间序列中的时间戳对其进行切片(即范围查询):

In [76]:
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [77]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
dtype: float64

In [79]:
ts.truncate(after='1/9/2011')

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
dtype: float64

In [80]:
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [ ]:
#面这些操作对 DataFrame 也有效。例如，对 DataFrame 的行进行索引:

In [81]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')     #每周三列出来
dates

DatetimeIndex(['2000-01-05', '2000-01-12', '2000-01-19', '2000-01-26',
               '2000-02-02', '2000-02-09', '2000-02-16', '2000-02-23',
               '2000-03-01', '2000-03-08', '2000-03-15', '2000-03-22',
               '2000-03-29', '2000-04-05', '2000-04-12', '2000-04-19',
               '2000-04-26', '2000-05-03', '2000-05-10', '2000-05-17',
               '2000-05-24', '2000-05-31', '2000-06-07', '2000-06-14',
               '2000-06-21', '2000-06-28', '2000-07-05', '2000-07-12',
               '2000-07-19', '2000-07-26', '2000-08-02', '2000-08-09',
               '2000-08-16', '2000-08-23', '2000-08-30', '2000-09-06',
               '2000-09-13', '2000-09-20', '2000-09-27', '2000-10-04',
               '2000-10-11', '2000-10-18', '2000-10-25', '2000-11-01',
               '2000-11-08', '2000-11-15', '2000-11-22', '2000-11-29',
               '2000-12-06', '2000-12-13', '2000-12-20', '2000-12-27',
               '2001-01-03', '2001-01-10', '2001-01-17', '2001-01-24',
      

In [84]:
long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas',
                                'New York', 'Ohio'])
long_df

,Colorado,Texas,New York,Ohio
2000-01-05,0.102827,0.400403,-0.936568,-0.063185
2000-01-12,1.842271,0.232059,0.664516,-0.329340
2000-01-19,-0.083667,-1.177878,0.253655,1.090384
2000-01-26,1.139249,0.494438,0.857712,-0.054764
2000-02-02,0.714331,1.268857,0.928245,0.117946
2000-02-09,-0.585345,-1.194708,1.012227,-0.495951
2000-02-16,0.191978,0.927557,-0.925906,-1.456259
2000-02-23,-1.785898,-0.204981,2.367215,-0.711614
2000-03-01,-1.299822,-1.265195,-0.095580,-0.684138
2000-03-08,0.109981,-0.030867,0.001860,0.686572


In [85]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,1.096297,1.180143,-0.723274,1.101502
2001-05-09,-1.148311,-0.338326,1.046686,0.519058
2001-05-16,0.242492,-2.370133,0.510788,-1.878959
2001-05-23,0.950015,-1.083047,0.541508,-1.015386
2001-05-30,1.357592,0.663670,-0.309027,-0.389945


## 带有重复索引的时间序列

In [ ]:
#在某些应用场景中，可能会存在多个观测数据落在同一个时间点上的情况。下面就是一个例子:

In [87]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000',
'1/2/2000',
                          '1/2/2000', '1/3/2000'])
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-02', '2000-01-02',
               '2000-01-03'],
              dtype='datetime64[ns]', freq=None)

In [88]:
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [89]:
#通过检查索引的 is_unique 属性，我们就可以知道它是不是唯一的:
dup_ts.index.is_unique      

False

In [ ]:
#对这个时间序列进行索引，要么产生标量值，要么产生切片，具体要看所选的时间点是否重复:

In [90]:
dup_ts['1/3/2000']  # not duplicated

4

In [91]:
dup_ts['1/2/2000']  # duplicated

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [93]:
#假设你想要对具有非唯一时间戳的数据进行聚合。一个办法是使用 groupby， 并传入 level=0:
grouped = dup_ts.groupby(level=0)      #levels：层及索引 
grouped

In [94]:
grouped.mean()     #求取均值

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [102]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

# 11.3 日期的范围、频率以及移动 P348

In [ ]:
#pandas 有一整套标准时间序列频率以及用于重 采样、频率推断、生成固定频率日期范围的工具。

In [ ]:
#例如，我们可以将之前那个 时间序列转换为一个具有固定频率(每日)的时间序列，只需调用 resample 即 可:

In [103]:
ts

2011-01-02    0.517332
2011-01-05   -1.372890
2011-01-07   -0.357989
2011-01-08    1.637554
2011-01-10    0.347589
2011-01-12    0.048733
dtype: float64

In [105]:
resampler = ts.resample('D')  #字符串“D”是每天的意思
resampler

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

## 生成日期范围

In [106]:
#pandas.date_range 可用于 根据指定的频率生成指定长度的 DatetimeIndex:
index = pd.date_range('2012-04-01', '2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [107]:
#默认情况下，date_range 会产生按天计算的时间点。如果只传入起始或结束日 期，那就还得传入一个表示一段时间的数字:
pd.date_range(start='2012-04-01', periods=20)    #传入起始时间，频率为20

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [108]:
pd.date_range(end='2012-06-01', periods=20)       ##传入结束时间，频率为20

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
#起始和结束日期定义了日期索引的严格边界。
#例如，如果你想要生成一个由每 月最后一个工作日组成的日期索引，可以传入"BM"频率(表示 business end of month，表 11-4 是频率列表)，
#这样就只会包含时间间隔内(或刚好在边界 上的)符合频率要求的日期:

In [109]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM')   #"BM"频率(表示 business end of month)由每月最后一个工作日

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [110]:
#表 11-4 基本的时间序列频率(不完整) P350

In [111]:
#date_range 默认会保留起始和结束时间戳的时间信息(如果有的话):
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [112]:
#虽然起始和结束日期带有时间信息，但你希望产生一组被规范化 (normalize)到午夜的时间戳。normalize 选项即可实现该功能:
pd.date_range('2012-05-02 12:56:31', periods=5,
normalize=True)   

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

## 频率和日期偏移量

In [115]:
#pandas 中的频率是由一个基础频率(base frequency)和一个乘数组成的。
#基 础频率通常以一个字符串别名表示，比如"M"表示每月，"H"表示每小时。
#对于 每个基础频率，都有一个被称为日期偏移量(date offset)的对象与之对应。
#例如，按小时计算的频率可以用 Hour 类表示:
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
hour

<Hour>

In [116]:
#传入一个整数即可定义偏移量的倍数:
four_hours = Hour(4)
four_hours

<4 * Hours>

In [117]:
#在基础频率前面放上一个整数即可创建倍数:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [118]:
#大部分偏移量对象都可通过加法进行连接:
Hour(2) + Minute(30)

<150 * Minutes>

In [119]:
#你也可以传入频率字符串(如"2h30min")，这种字符串可以被高效地解 析为等效的表达式:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

In [ ]:
#"M"(日历月末)和"BM" (每月最后一个工作日)就取决于每月的天数，对于后者，还要考虑月末是不 是周末。
#由于没有更好的术语，我将这些称为锚点偏移量(anchored offset)。

## WOM 日期

In [121]:
#WOM(Week Of Month)是一种非常实用的频率类，它以 WOM 开头。它使你能获 得诸如“每月第 3 个星期五”之类的日期:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

In [122]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

## 移动(超前和滞后)数据

In [123]:
#移动(shifting)指的是沿着时间轴将数据前移或后移。
#Series 和 DataFrame 都有一个 shift 方法用于执行单纯的前移或后移操作，保持索引不变:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, freq='M'))
ts

2000-01-31   -0.566351
2000-02-29    1.839052
2000-03-31   -0.914734
2000-04-30    0.544968
Freq: M, dtype: float64

In [124]:
ts.shift(2)      #顺移2个单位

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.566351
2000-04-30    1.839052
Freq: M, dtype: float64

In [125]:
ts.shift(-2)      #逆移2个单位

2000-01-31   -0.914734
2000-02-29    0.544968
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

In [ ]:
#shift 通常用于计算一个时间序列或多个时间序列(如 DataFrame 的列)中的 百分比变化。可以这样表达:
ts / ts.shift(1) - 1

In [131]:
#由于单纯的移位操作不会修改索引，所以部分数据会被丢弃。
#因此，如果频率 已知，则可以将其传给 shift 以便实现对时间戳进行位移而不是对数据进行简 单位移:
ts

2000-01-31   -0.566351
2000-02-29    1.839052
2000-03-31   -0.914734
2000-04-30    0.544968
Freq: M, dtype: float64

In [127]:
ts.shift(2, freq='M')    #相当于对时间戳的移动

2000-03-31   -0.566351
2000-04-30    1.839052
2000-05-31   -0.914734
2000-06-30    0.544968
Freq: M, dtype: float64

In [132]:
ts.shift(3, freq='D')     #相当于对时间戳的移动 ，移动3天

2000-02-03   -0.566351
2000-03-03    1.839052
2000-04-03   -0.914734
2000-05-03    0.544968
dtype: float64

In [133]:
ts.shift(1, freq='90T')       # #相当于对时间戳的移动 ，移动90分钟

2000-01-31 01:30:00   -0.566351
2000-02-29 01:30:00    1.839052
2000-03-31 01:30:00   -0.914734
2000-04-30 01:30:00    0.544968
Freq: M, dtype: float64

## 通过偏移量对日期进行位移

In [134]:
#pandas 的日期偏移量还可以用在 datetime 或 Timestamp 对象上:
from pandas.tseries.offsets import Day, MonthEnd

In [135]:
now = datetime(2011, 11, 17)
now

datetime.datetime(2011, 11, 17, 0, 0)

In [136]:
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [ ]:
#如果加的是锚点偏移量(比如 MonthEnd)，第一次增量会将原日期向前滚动到 符合频率规则的下一个日期:

In [137]:
now + MonthEnd()    #第一次增量会将原日期向前滚动到 符合频率规则的下一个日期

Timestamp('2011-11-30 00:00:00')

In [138]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

In [ ]:
#通过锚点偏移量的 rollforward 和 rollback 方法，可明确地将日期向前或向后 “滚动”:

In [139]:
offset = MonthEnd()    #偏移量

In [140]:
offset.rollforward(now)     #向后滚动

Timestamp('2011-11-30 00:00:00')

In [142]:
offset.rollback(now)     ##向前滚动

Timestamp('2011-10-31 00:00:00')

In [154]:
#日期偏移量还有一个巧妙的用法，即结合 groupby 使用这两个“滚动”方法:
ts = pd.Series(np.random.randn(20),
               index=pd.date_range('1/15/2000', periods=20, freq='4d'))
ts

2000-01-15    0.167977
2000-01-19   -0.071823
2000-01-23    0.453260
2000-01-27   -0.755439
2000-01-31    0.995263
2000-02-04   -0.600792
2000-02-08   -0.461316
2000-02-12   -0.514329
2000-02-16   -0.070487
2000-02-20    0.000965
2000-02-24    0.030494
2000-02-28   -1.042908
2000-03-03   -1.505139
2000-03-07    0.314823
2000-03-11    1.220977
2000-03-15   -0.332140
2000-03-19   -1.000860
2000-03-23    1.336212
2000-03-27   -0.297063
2000-03-31   -0.376657
Freq: 4D, dtype: float64

In [155]:
ts.groupby(offset.rollforward).mean()     #对各个月份去平均值，作为该月最后工作日的值。

2000-01-31    0.157847
2000-02-29   -0.379768
2000-03-31   -0.079981
dtype: float64

In [ ]:
#更简单、更快速地实现该功能的办法是使用 resample(11.6 小节将对此 进行详细介绍):

In [156]:
ts.resample('M').mean()

2000-01-31    0.157847
2000-02-29   -0.379768
2000-03-31   -0.079981
Freq: M, dtype: float64

# 11.4 时区处理 P358

In [ ]:
#时区是以 UTC 偏移量的形式表示的。

In [157]:
import pytz
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [ ]:
#要从 pytz 中获取时区对象，使用 pytz.timezone 即可:

In [159]:
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

## 时区本地化和转换

In [160]:
#默认情况下，pandas 中的时间序列是单纯的(naive)时区。看看下面这个时 间序列:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
rng

DatetimeIndex(['2012-03-09 09:30:00', '2012-03-10 09:30:00',
               '2012-03-11 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00'],
              dtype='datetime64[ns]', freq='D')

In [161]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00   -0.465987
2012-03-10 09:30:00   -0.652920
2012-03-11 09:30:00   -1.328444
2012-03-12 09:30:00   -1.995042
2012-03-13 09:30:00    1.863015
2012-03-14 09:30:00   -2.062446
Freq: D, dtype: float64

In [163]:
#其索引的 tz 字段为 None:
print(ts.index.tz)

None


In [164]:
#可以用时区集生成日期范围:
pd.date_range('3/9/2012 9:30', periods=10, freq='D',
tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [165]:
#从单纯到本地化的转换是通过 tz_localize 方法处理的:
ts

2012-03-09 09:30:00   -0.465987
2012-03-10 09:30:00   -0.652920
2012-03-11 09:30:00   -1.328444
2012-03-12 09:30:00   -1.995042
2012-03-13 09:30:00    1.863015
2012-03-14 09:30:00   -2.062446
Freq: D, dtype: float64

In [166]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -0.465987
2012-03-10 09:30:00+00:00   -0.652920
2012-03-11 09:30:00+00:00   -1.328444
2012-03-12 09:30:00+00:00   -1.995042
2012-03-13 09:30:00+00:00    1.863015
2012-03-14 09:30:00+00:00   -2.062446
Freq: D, dtype: float64

In [167]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [168]:
#一旦时间序列被本地化到某个特定时区，就可以用 tz_convert 将其转换到别的 时区了:
ts_utc.tz_convert('America/New_York')   #tz_convert转换时区

2012-03-09 04:30:00-05:00   -0.465987
2012-03-10 04:30:00-05:00   -0.652920
2012-03-11 05:30:00-04:00   -1.328444
2012-03-12 05:30:00-04:00   -1.995042
2012-03-13 05:30:00-04:00    1.863015
2012-03-14 05:30:00-04:00   -2.062446
Freq: D, dtype: float64

In [169]:
#对于上面这种时间序列(它跨越了美国东部时区的夏令时转变期)，我们可以 将其本地化到 EST，
#然后转换为 UTC 或柏林时间:
ts_eastern = ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')          #转变纽约时区

2012-03-09 14:30:00+00:00   -0.465987
2012-03-10 14:30:00+00:00   -0.652920
2012-03-11 13:30:00+00:00   -1.328444
2012-03-12 13:30:00+00:00   -1.995042
2012-03-13 13:30:00+00:00    1.863015
2012-03-14 13:30:00+00:00   -2.062446
Freq: D, dtype: float64

In [170]:
ts_eastern.tz_convert('Europe/Berlin')      ##转变柏林时区

2012-03-09 15:30:00+01:00   -0.465987
2012-03-10 15:30:00+01:00   -0.652920
2012-03-11 14:30:00+01:00   -1.328444
2012-03-12 14:30:00+01:00   -1.995042
2012-03-13 14:30:00+01:00    1.863015
2012-03-14 14:30:00+01:00   -2.062446
Freq: D, dtype: float64

In [171]:
#tz_localize 和 tz_convert 也是 DatetimeIndex 的实例方法:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

## 操作时区意识型 Timestamp 对象

In [173]:
#跟时间序列和日期范围差不多，独立的 Timestamp 对象也能被从单纯型 (naive)本地化为时区意识型(time zone-aware)，
#并从一个时区转换到另 一个时区:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp

Timestamp('2011-03-12 04:00:00')

In [175]:
stamp_utc = stamp.tz_localize('utc')
stamp_utc

Timestamp('2011-03-12 04:00:00+0000', tz='UTC')

In [176]:
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

In [178]:
#在创建 Timestamp 时，还可以传入一个时区信息:
stamp_moscow = pd.Timestamp('2011-03-12 04:00',
tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [179]:
#时区意识型 Timestamp 对象在内部保存了一个 UTC 时间戳值(自 UNIX 纪元 (1970 年 1 月 1 日)算起的纳秒数)。
#这个 UTC 值在时区转换过程中是不会发 生变化的:
stamp_utc.value

1299902400000000000

In [180]:
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

In [ ]:
#当使用 pandas 的 DateOffset 对象执行时间算术运算时，
#运算过程会自动关注 是否存在夏令时转变期。

In [181]:
#我们创建了在 DST 转变之前的时间戳。首先， 来看夏令时转变前的 30 分钟:
from pandas.tseries.offsets import Hour
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [182]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [183]:
#然后，夏令时转变前 90 分钟:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [184]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

## 不同时区之间的运算

In [185]:
#如果两个时间序列的时区不同，在将它们合并到一起时，最终结果就会是 UTC。
#由于时间戳其实是以 UTC 存储的，所以这是一个很简单的运算，并不需要 发生任何转换:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')  #freq='B'每工作日
rng

DatetimeIndex(['2012-03-07 09:30:00', '2012-03-08 09:30:00',
               '2012-03-09 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00',
               '2012-03-15 09:30:00', '2012-03-16 09:30:00',
               '2012-03-19 09:30:00', '2012-03-20 09:30:00'],
              dtype='datetime64[ns]', freq='B')

In [186]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-07 09:30:00   -2.014382
2012-03-08 09:30:00    0.363347
2012-03-09 09:30:00    0.324812
2012-03-12 09:30:00    1.978397
2012-03-13 09:30:00   -1.525933
2012-03-14 09:30:00    0.039862
2012-03-15 09:30:00    0.022068
2012-03-16 09:30:00    1.975204
2012-03-19 09:30:00   -1.906401
2012-03-20 09:30:00    1.159852
Freq: B, dtype: float64

In [187]:
ts1 = ts[:7].tz_localize('Europe/London')
ts1

2012-03-07 09:30:00+00:00   -2.014382
2012-03-08 09:30:00+00:00    0.363347
2012-03-09 09:30:00+00:00    0.324812
2012-03-12 09:30:00+00:00    1.978397
2012-03-13 09:30:00+00:00   -1.525933
2012-03-14 09:30:00+00:00    0.039862
2012-03-15 09:30:00+00:00    0.022068
Freq: B, dtype: float64

In [188]:
ts2 = ts1[2:].tz_convert('Europe/Moscow')
ts2

2012-03-09 13:30:00+04:00    0.324812
2012-03-12 13:30:00+04:00    1.978397
2012-03-13 13:30:00+04:00   -1.525933
2012-03-14 13:30:00+04:00    0.039862
2012-03-15 13:30:00+04:00    0.022068
Freq: B, dtype: float64

In [190]:
result = ts1 + ts2
result

2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00    0.649624
2012-03-12 09:30:00+00:00    3.956795
2012-03-13 09:30:00+00:00   -3.051866
2012-03-14 09:30:00+00:00    0.079724
2012-03-15 09:30:00+00:00    0.044136
Freq: B, dtype: float64

In [191]:
result.index    #不同时区计算结果还是UTC时区

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

# 11.5 时期及其算术运算 P364

In [192]:
#时期(period)表示的是时间区间，比如数日、数月、数季、数年等。
#Period 类所表示的就是这种数据类型，其构造函数需要用到一个字符串或整数，以及 表 11-4 中的频率:
p = pd.Period(2007, freq='A-DEC')   #freq='A-DEC' 每年指定月份的最后一个日历日
p

Period('2007', 'A-DEC')

In [193]:
#Period对象表示的是从2007年1月1日到2007年12月31日之间 的整段时间。
#只需对 Period 对象加上或减去一个整数即可达到根据其频率进行 位移的效果:
p + 5

Period('2012', 'A-DEC')

In [194]:
p - 2

Period('2005', 'A-DEC')

In [195]:
#如果两个 Period 对象拥有相同的频率，则它们的差就是它们之间的单位数量:
pd.Period('2014', freq='A-DEC') - p

<7 * YearEnds: month=12>

In [196]:
#period_range 函数可用于创建规则的时期范围:
rng = pd.period_range('2000-01-01', '2000-06-30', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [197]:
#PeriodIndex 类保存了一组 Period，它可以在任何 pandas 数据结构中被用作轴 索引:
pd.Series(np.random.randn(6), index=rng)

2000-01    0.745979
2000-02    2.097737
2000-03   -1.466166
2000-04    0.714979
2000-05   -1.203228
2000-06    1.007026
Freq: M, dtype: float64

In [198]:
#如果你有一个字符串数组，你也可以使用 PeriodIndex 类:
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')     #freq='Q-DEC'对于指定月份的季度最后一月的最后一个工作日
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

## 时期的频率转换

In [199]:
#Period 和 PeriodIndex 对象都可以通过其 asfreq 方法被转换成别的频率。
#假设我们有一个年度时期，希望将其转换为当年年初或年末的一个月度时期。该 任务非常简单:
p = pd.Period('2007', freq='A-DEC')
p

Period('2007', 'A-DEC')

In [200]:
p.asfreq('M', how='start')  #转化成年初时刻

Period('2007-01', 'M')

In [201]:
p.asfreq('M', how='end')    #转化成年末时刻

Period('2007-12', 'M')

In [203]:
#你可以将 Period('2007','A-DEC')看做一个被划分为多个月度时期的时间段中 的游标。
#对于一个不以 12 月结束的财政年度，月度 子时期的归属情况就不一样了:
p = pd.Period('2007', freq='A-JUN')
p    #将p划到中断标轴，以下一个月为开始

Period('2007', 'A-JUN')

In [204]:
p.asfreq('M', 'start')

Period('2006-07', 'M')

In [205]:
p.asfreq('M', 'end')

Period('2007-06', 'M')

In [206]:
#在将高频率转换为低频率时，超时期(superperiod)是由子时期 (subperiod)所属的位置决定的。
#在 A-JUN 频率中，月份“2007 年 8 月”实际上是属于周期“2008 年”的:
p = pd.Period('Aug-2007', 'M')
p

Period('2007-08', 'M')

In [207]:
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

In [208]:
#完整的 PeriodIndex 或 TimeSeries 的频率转换方式也是如此:
rng = pd.period_range('2006', '2009', freq='A-DEC')
rng

PeriodIndex(['2006', '2007', '2008', '2009'], dtype='period[A-DEC]', freq='A-DEC')

In [209]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2006    0.444722
2007   -0.161575
2008    0.583363
2009   -0.636277
Freq: A-DEC, dtype: float64

In [210]:
ts.asfreq('M', how='start')      #以年初作为频率

2006-01    0.444722
2007-01   -0.161575
2008-01    0.583363
2009-01   -0.636277
Freq: M, dtype: float64

In [211]:
#根据年度时期的第一个月，每年的时期被取代为每月的时期。
#如果我们 想要每年的最后一个工作日，我们可以使用“B”频率，并指明想要该时期的末 尾:
ts.asfreq('B', how='end')      #“B”频率 每年的最后一个工作日

2006-12-29    0.444722
2007-12-31   -0.161575
2008-12-31    0.583363
2009-12-31   -0.636277
Freq: B, dtype: float64

## 按季度计算的时期频率

In [ ]:
+

## 将 Timestamp 转换为 Period(及其反向过程)

## 通过数组创建 PeriodIndex

# 11.6 重采样及频率转换 P374

## 降采样

## OHLC 重采样

## 升采样和插值

## 通过时期进行重采样

# 11.7 移动窗口函数 P381

## 指数加权函数

## 二元移动窗口函数

## 用户定义的移动窗口函数

# 11.8 总结 P389